In [1]:
import os
os.chdir('..')

In [2]:
from model.network import Converter
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)

In [5]:
import os
import pickle
from pathlib import Path

import numpy as np
import tensorflow as tf
import lpips_tf

from keras import backend as K
from keras import optimizers, losses, regularizers
from keras.layers import Conv2D, Dense, UpSampling2D, LeakyReLU, Activation
from keras.layers import Layer, Input, Reshape, Lambda, Flatten, Concatenate, Embedding, GaussianNoise
from keras.models import Model, load_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from keras.applications import vgg16
from keras_lr_multiplier import LRMultiplier
from assets import AssetManager

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks.callbacks import EarlyStopping, CSVLogger
from keras.callbacks.tensorboard_v1 import TensorBoard

from sklearn.preprocessing import OneHotEncoder

In [8]:
class LORDContentClassifier:
    def __init__(self, subset=None,
                 base_dir = 'results', model_name = 'minst_10_model', data_name = 'minst_10_test'):
        assets = AssetManager(base_dir)
        data = np.load(assets.get_preprocess_file_path(data_name))
        imgs, classes, contents, n_classes = data['imgs'], data['classes'], data['contents'], data['n_classes']
        imgs = imgs.astype(np.float32) / 255.0
        if subset is not None:
            self.curr_imgs = imgs[:subset]
            self.classes = classes[:subset]
        else:
            self.curr_imgs = imgs
            self.classes = classes

        self.onehot_enc = OneHotEncoder()
        self.onehot_classes = self.onehot_enc.fit_transform(self.classes.reshape(-1,1))

        self.n_images = self.curr_imgs.shape[0]
        
        self.converter = Converter.load( assets.get_model_dir(model_name), include_encoders=True)
        self.content_codes = self.converter.content_encoder.predict(self.curr_imgs)
        class_codes = self.converter.class_encoder.predict(self.curr_imgs)
        class_adain_params = self.converter.class_modulation.predict(class_codes)
        self.class_adain_params = class_adain_params.reshape(class_adain_params.shape[0], -1)
        
    def train_content_classifier(self, n_epochs):        
        model = self.get_model(self.content_codes.shape[1])        
        callbacks = [EarlyStopping(), CSVLogger('LORDContentClassifier_content.csv'), TensorBoard()]
        model.fit(self.content_codes, self.onehot_classes, epochs=20, validation_split=0.3, callbacks=callbacks)
        
    def train_class_classifier(self, n_epochs):        
        model = self.get_model(self.class_adain_params.shape[1])        
        callbacks = [EarlyStopping(), CSVLogger('LORDContentClassifier_class.csv'), TensorBoard()]
        model.fit(self.class_adain_params, self.onehot_classes, epochs=20, validation_split=0.3, callbacks=callbacks)
        
    def get_model(self, input_dim):
        model = Sequential()
        model.add(Dense(units=256, activation='relu', input_dim=input_dim))
        model.add(Dense(units=10, activation='softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        return model

In [9]:
cc = LORDContentClassifier()
# cc.train_content_classifier(10000)
# cc.train_class_classifier(10000)

loading models...


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '



vgg arch:
Model: "vgg"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 64, 64, 3)         0         
_________________________________________________________________
vgg_normalization_1 (VggNorm (None, 64, 64, 3)         0         
_________________________________________________________________
model_1 (Model)              [(None, 64, 64, 64), (Non 14714688  
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________



KeyboardInterrupt: 